In [1]:
from pathlib import Path
import sys  
import os

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "libs")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

In [13]:
from libs.Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
import numpy as np
from operator import itemgetter

In [4]:
from Utils.load_URM import load_URM
from Utils.load_ICM import load_ICM
from libs.Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_all = load_URM("../data/data_train.csv")
ICM_all = load_ICM("../data/data_ICM_metadata.csv")

In [5]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 164 ( 0.5%) Users that have less than 1 test interactions


In [24]:
np.linspace(0, 2, 20)

array([0.        , 0.10526316, 0.21052632, 0.31578947, 0.42105263,
       0.52631579, 0.63157895, 0.73684211, 0.84210526, 0.94736842,
       1.05263158, 1.15789474, 1.26315789, 1.36842105, 1.47368421,
       1.57894737, 1.68421053, 1.78947368, 1.89473684, 2.        ])

In [13]:
#from Data_manager.Movielens.Movielens10MReader import Movielens10MReader

In [6]:
#data_reader = Movielens10MReader()
#data_loaded = data_reader.load_data()

#URM_all = data_loaded.get_URM_all()

In [17]:
#URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [22]:
URM_train

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 8000043 stored elements and shape (69878, 10681)>

In [ ]:
#MAP = []
URM_train_coo = URM_train.tocoo()


for topK_ in [50, 100, 150, 200]:
    for l1_ratio_ in np.linspace(0.001, 1, 10):
        for alpha_ in np.linspace(0.00001, 0.0005, 3):
            slim_model = SLIMElasticNetRecommender(URM_train_coo, verbose=True)
            slim_model.fit(l1_ratio=l1_ratio_, alpha = alpha_, topK = topK_)

            result_dict, _ = evaluator_validation.evaluateRecommender(slim_model)
            MAP.append(('topK={}'.format(topK_), 'l1_ratio={}'.format(l1_ratio_), 'alpha={}'.format(alpha_), result_dict["MAP"][10]))

            print(MAP[-1])
            

In [14]:
max_tuple = max(MAP, key=itemgetter(3))
print('The best combination for the ICF recommender is ({},{},{}), with a MAP = {}'.format(max_tuple[0],max_tuple[1], max_tuple[2], max_tuple[3]))

The best combination for the ICF recommender is (topK=20,l1_ratio=0.15873684210526315,alpha=0.00034), with a MAP = 0.05992418684929483


In [ ]:
slim_model = SLIMElasticNetRecommender(URM_train)

In [14]:
slim_model.fit()

SLIMElasticNetRecommender: Processed 38121 (100.0%) in 3.09 min. Items per second: 205.70


In [16]:
result_dict, _ = evaluator_validation.evaluateRecommender(slim_model)

EvaluatorHoldout: Processed 35590 (100.0%) in 8.42 sec. Users per second: 4225


In [17]:
result_dict

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.000865,0.001183,0.000963,0.000252,0.000365,0.002385,0.001031,0.000912,0.008205,0.00245,...,0.995914,0.008171,0.995914,0.000638,4.95516,0.955743,0.00105,0.335671,2.365443,0.232804


In [18]:
result_dict["MAP"][10]

0.00025152753805465186